## **`Paso 4.8:` Modificación final al proceso de ingesta desde HTTP donde configuramos el pipeline para ingestar multiples archivos con solo una ejecución**

### **1. Creación del Dataset de origen `ds_ecdc_file_list` que hace referencia al archivo `ecdc_file_list.json` alojado en el contenedor `configs` en Azure Blob Storage**

#### **Paso 1: Crear un nuevo dataset**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Datasets"**: En el panel de **Author**, haz clic en **Datasets**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Datasets**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New dataset"**: En el menú desplegable, selecciona **New dataset** para crear un nuevo dataset.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p831.png](https://i.postimg.cc/xdS4vphv/p831.png)](https://postimg.cc/9DJbVbW0)

#### **Paso 2: Configurar el nuevo dataset**

1. **Elegir tipo de dataset**: En la ventana de selección, busca y selecciona **Azure Blob Storage** como el tipo de datastore.
2. **Hacer clic en "Continue"**: Después de seleccionar HTTP, haz clic en **Continue** para proceder.

[![p832.png](https://i.postimg.cc/hGb3pbRH/p832.png)](https://postimg.cc/d73B0C1B)

#### **Paso 3: Configurar el formato del dataset**

1. **Seleccionar el formato**: En el campo de selección de formato, elige **JSON**.
2. **Hacer clic en "Continue"**: Después de seleccionar el formato, haz clic en **Continue**.

[![p833.png](https://i.postimg.cc/1tv7ZmtB/p833.png)](https://postimg.cc/qhCxsd26)

#### **Paso 4: Configurar las propiedades del dataset**

1. **Nombre del dataset**: En el campo de nombre, ingresa `ds_ecdc_file_list`.
2. **Linked Service**: En el menú desplegable, selecciona el Linked Service que creaste previamente, `ls_ablob_covidreportingsa`.
3. **File path**: Indica la ruta Indiquemos el contenedor `configs` e ingresa `ecdc_file_list.json`.
4. **Import schema**: Selecciona `From connection/store` para importar el schema.
6. **Guardar el dataset**: Haz clic en el botón **OK** para almacenar el nuevo dataset.

[![p834.png](https://i.postimg.cc/C1hrtBFd/p834.png)](https://postimg.cc/t1czngnH)

#### **Paso 5: Verificar el dataset**

1. **Revisar la lista de datasets**: Deberías ver tu nuevo dataset `ds_ecdc_file_list` en la lista de datasets. Puedes hacer clic en él para revisar su configuración.

[![p835.png](https://i.postimg.cc/6QYf8HX5/p835.png)](https://postimg.cc/7CJ2cnhp)

### **2. Actualizar el Pipeline de ingesta de datos parametrizado `pl_ingest_ecdc_data` el cual contendrá las actividades `Lookup`, `For Each` y `Copy data`**

El pipeline `pl_ingest_ecdc_data` actualmente contiene una sola actividad: `Copy Data`. Vamos a añadir nuevas actividades, que incluirán:

- `Lookup`
- `For Each`

#### **Paso 1: Configurar la Actividad `Lookup`**

1. **Buscar la actividad "Lookup"**: En el panel de actividades a la izquierda, busca `Lookup` en la sección de `General`.
2. **Arrastrar la actividad al canvas**: Arrastra la actividad `Lookup` al canvas del pipeline.
3. **Seleccionar la actividad "Lookup"**: Haz clic en la actividad que agregaste para abrir el panel de configuración.
4. **Pestaña General**:
   - En **Name**, ingresa `Lookup ECDC File List`.
 
3. **Pestaña Settings**:

   - En **Source dataset** selecciona `ds_ecdc_file_list`. 
   - En **First row only** desactivar la casilla.

[![p836.png](https://i.postimg.cc/qRpB5Lhn/p836.png)](https://postimg.cc/hJ3qJV0P)
[![p837.png](https://i.postimg.cc/PxKfswY7/p837.png)](https://postimg.cc/gXXb3rMy)

**INPUT**

```json
{
    "source": {
        "type": "JsonSource",
        "storeSettings": {
            "type": "AzureBlobStorageReadSettings",
            "recursive": true,
            "enablePartitionDiscovery": false
        },
        "formatSettings": {
            "type": "JsonReadSettings"
        }
    },
    "dataset": {
        "referenceName": "ds_ecdc_file_list",
        "type": "DatasetReference",
        "parameters": {}
    },
    "firstRowOnly": false
}
```

**OUTPUT**

```json
{
	"count": 4,
	"value": [
		{
			"sourceBaseURL": "https://github.com",
			"sourceRelativeURL": "perezlino/data_fake/raw/main/adf2/cases_deaths.csv",
			"sinkFileName": "cases_deaths.csv"
		},
		{
			"sourceBaseURL": "https://github.com",
			"sourceRelativeURL": "perezlino/data_fake/raw/main/adf2/hospital_admissions.csv",
			"sinkFileName": "hospital_admissions.csv"
		},
		{
			"sourceBaseURL": "https://github.com",
			"sourceRelativeURL": "perezlino/data_fake/raw/main/adf2/testing.csv",
			"sinkFileName": "testing.csv"
		},
		{
			"sourceBaseURL": "https://github.com",
			"sourceRelativeURL": "perezlino/data_fake/raw/main/adf2/country_response.csv",
			"sinkFileName": "country_response.csv"
		}
	],
	"effectiveIntegrationRuntime": "AutoResolveIntegrationRuntime (East US)",
	"billingReference": {
		"activityType": "PipelineActivity",
		"billableDuration": [
			{
				"meterType": "AzureIR",
				"duration": 0.016666666666666666,
				"unit": "Hours"
			}
		],
		"totalBillableDuration": [
			{
				"meterType": "AzureIR",
				"duration": 0.016666666666666666,
				"unit": "Hours"
			}
		]
	},
	"durationInQueue": {
		"integrationRuntimeQueue": 0
	}
}
```

#### **Paso 2: Configurar la Actividad `For Each`**

1. **Buscar la actividad "For Each"**: En el panel de actividades a la izquierda, busca `For Each` en la sección de `Iteration & conditionals`.
2. **Arrastrar la actividad al canvas**: Arrastra la actividad `For Each` al canvas del pipeline.
3. **Seleccionar la actividad "For Each"**: Haz clic en la actividad que agregaste para abrir el panel de configuración.
4. **Pestaña General**:
   - En **Name**, ingresa `Execute Copy For Every Record`.
5. **Pestaña Settings**:
   - En **Items** indica `@activity('Lookup ECDC File List').output.value`. 
6. Cortamos la actividad `Copy data` que teniamos en el pipeline y la pegamos dentro de la actividad `For Each`.
7. Borramos todos los parámetros a nivel de pipeline, ya que, no los utilizaremos.
8. Para los parámetros del dataset de origen **ds_ecdc_raw_csv_http**: `relativeURL` y `baseURL`, le pasamos como valor dinámico lo obtenido en **items**, es decir, los valores de `sourceBaseURL` y `sourceRelativeURL`.
9. Para lel parámetro del dataset de destino **ds_ecdc_raw_csv_dl**: `fileName` le pasamos como valor dinámico lo obtenido en **items**, es decir, el valor de `sinkFileName`.

[![p838.png](https://i.postimg.cc/k5TMLSxp/p838.png)](https://postimg.cc/CZfVqzrG)
[![p839.png](https://i.postimg.cc/50j9JtgN/p839.png)](https://postimg.cc/CBTpHS9W)
[![p841.png](https://i.postimg.cc/fy9w3g1G/p841.png)](https://postimg.cc/dZJMpns6)
[![p842.png](https://i.postimg.cc/mhWT7yst/p842.png)](https://postimg.cc/n9T8J7MJ)
[![p843.png](https://i.postimg.cc/L4cPyd4Z/p843.png)](https://postimg.cc/Thchw4DR)
[![p844.png](https://i.postimg.cc/8kJWBkGS/p844.png)](https://postimg.cc/kDCB7mD1)
[![p845.png](https://i.postimg.cc/bN1tW6dV/p845.png)](https://postimg.cc/vcHT6tr5)
[![p846.png](https://i.postimg.cc/sgyhrphj/p846.png)](https://postimg.cc/6yj3V4cg)
[![p847.png](https://i.postimg.cc/9QpqFrdG/p847.png)](https://postimg.cc/XrrvsNDJ)
[![p848.png](https://i.postimg.cc/jjSnWtcB/p848.png)](https://postimg.cc/K1C8wXd5)
[![p849.png](https://i.postimg.cc/N0C9mSPF/p849.png)](https://postimg.cc/1g63ZCS1)
[![p850.png](https://i.postimg.cc/XJZZ8LT8/p850.png)](https://postimg.cc/phHXVfVh)
[![p851.png](https://i.postimg.cc/FR2YncvQ/p851.png)](https://postimg.cc/CBsMRRdv)
[![p852.png](https://i.postimg.cc/hvTXFj4Q/p852.png)](https://postimg.cc/xNjfM9z9)
[![p853.png](https://i.postimg.cc/sfhQGnPf/p853.png)](https://postimg.cc/7bqYpnQj)

#### **Paso 3: Validar el Pipeline**

1. **Validar la Configuración**: En la parte superior del panel, haz clic en el botón **"Validate"** para comprobar que no haya errores en la configuración del pipeline.

[![p854.png](https://i.postimg.cc/nLNVHJP4/p854.png)](https://postimg.cc/WD6cnQjz)
[![p855.png](https://i.postimg.cc/yd91xbMw/p855.png)](https://postimg.cc/62WJ1Y50)

#### **Paso 4: Ejecutar el Pipeline**

1. **Ejecutar el Pipeline**: Haz clic en el botón **"Debug"** en la parte superior para ejecutar el pipeline en modo de depuración.

[![p856.png](https://i.postimg.cc/L68c0Wy2/p856.png)](https://postimg.cc/WtxYd5LH)

#### **Paso 5: Revisar los Resultados**

1. Verificamos los resultados.

[![p857.png](https://i.postimg.cc/CxZWwqT2/p857.png)](https://postimg.cc/jDb8XCj6)

#### **Paso 6: Verificar la creación de los archivos en la ruta `raw/ecdc` en el ADLS**

1. Nos dirigimos al ADLS y revisamos la ruta **raw/ecdc** para ver si se ingestaron los archivos: 

    - `cases_deaths.csv`
    - `hospital_admissions.csv`
    - `testing.csv`
    - `country_response.csv` 

[![p858.png](https://i.postimg.cc/TYNz8cSm/p858.png)](https://postimg.cc/VdMGnXzL)